In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
from tqdm.autonotebook import tqdm
from time import sleep

data_dir = "./jdata/"
print(os.listdir(data_dir))
tqdm.pandas(desc="apply", leave=False)

['jdata_user.csv', 'jdata_shop.csv', '说明.txt', 'jdata_comment.csv', 'jdata_action.csv', 'jdata_product.csv']


/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Intuitive: Use the data from previous one and a half months to predict the targeted week

In [2]:
user_df = pd.read_csv(data_dir + 'jdata_user.csv')
shop_df = pd.read_csv(data_dir + 'jdata_shop.csv')
comment_df = pd.read_csv(data_dir + 'jdata_comment.csv')
action_df = pd.read_csv(data_dir + 'jdata_action.csv')
product_df = pd.read_csv(data_dir + 'jdata_product.csv')

In [3]:
# returns True if a is earlier than b
# params: a(month, day), b(month, day)
def compare_date(a: (int, int), b: (int, int)):
    if a[0] > b[0]:
        return False
    if a[0] < b[0]:
        return True
    return a[1] < b[1]

month_days = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# returns the number of days in between a and b including b
def date_diff(a: (int, int), b: (int, int)):
    if not compare_date(a, b):
        a, b = b, a
    if a[0] == b[0]:
        return b[1] - a[1]
    result = 0
    curr = b[0] - 1
    while curr != a[0]:
        result += month_days[curr]
        curr -= 1
    result += b[1]
    result += (month_days[a[0]] - a[1])
    return result

In [4]:
tqdm.pandas(desc="year", leave=False)
action_df['action_year'] = action_df['action_time'].progress_apply(lambda t: int(t[:4]))
tqdm.pandas(desc="month", leave=False)
action_df['action_month'] = action_df['action_time'].progress_apply(lambda t: int(t[5:7]))
tqdm.pandas(desc="time", leave=False)
action_df['action_day'] = action_df['action_time'].progress_apply(lambda t: int(t[8:10]))
action_df = action_df.sort_values(by=['action_year', 'action_month', 'action_day'])

In [5]:
action_df.reset_index(drop=True, inplace=True)

In [6]:
tqdm.pandas(desc="date", leave=False)
action_df['action_date'] = action_df[['action_month', 'action_day']].progress_apply(lambda t: (t[0], t[1]), axis=1, raw=True)

In [15]:
tqdm.pandas(desc="days since", leave=False)
action_df['action_days_since'] = action_df['action_date'].progress_apply(lambda t: date_diff(t, (2, 1)))

In [8]:
purchase_df = action_df[action_df['type'] == 2].merge(product_df[['sku_id', 'shop_id', 'cate']], on='sku_id', how='left')

In [12]:
user_df['reg_year'] = user_df['user_reg_tm'].apply(lambda t: int(t[:4]))

In [21]:
gc.collect()

945

20

In [14]:
# Periods: [(train_start, train_end, label_start, label_end), ...]
periods = [((2, 1), (3, 18), (3, 19), (3, 25)),
           ((2, 14), (3, 31), (4, 1), (4, 7)),
           ((2, 22), (4, 8), (4, 9), (4, 15))]
test_periods = ((3, 1), (4, 15), (4, 16), (4, 22))

In [ ]:
def select_training_from_period(start, end):
    